In [1]:
setwd('/lustre/scratch117/cellgen/team297/kt16/COVID_imperial_renal')

In [2]:
suppressPackageStartupMessages(library(SingleCellExperiment))
suppressPackageStartupMessages(library(edgeR))
suppressPackageStartupMessages(library(scran))
suppressPackageStartupMessages(library(scater))
suppressPackageStartupMessages(library(glmmSeq))

In [3]:
sce <- readRDS('h5ad/df.fil3_gex_bcells_vdj_sce_B_ASC_IgG.RDS')
counts(sce) <- assays(sce)[['X']]
sce$WHO_severity <- factor(sce$WHO_severity, levels = c('NA', 'mild', 'moderate', 'severe', 'critical'))

# Remove samples with less than 5 cells
nCells <- table(sce$sample_id)
rmSamples <- names(nCells[nCells<5])
sce <- sce[,!sce$sample_id %in% rmSamples]
# Summarize Counts
smrzd <- aggregateAcrossCells(sce, id=as.character(colData(sce)[,c("sample_id")]))
y <- DGEList(counts=counts(smrzd), samples=colData(smrzd))
keep <- filterByExpr(y, group=y$samples$case_control, min.count=3, min.total.count=5)
y <- y[keep,]
# ensure the factor levels are correct
y$samples$case_control <- droplevels(y$samples$case_control)
y$samples$WHO_severity <- droplevels(y$samples$WHO_severity)
y$samples$WHO_severity <- ordered(y$samples$WHO_severity) # ordered it so that i can test linear/quadratic trends in the model
# Estimate Dispersion
disp  <- suppressMessages(setNames(edgeR::estimateDisp(y)$tagwise.dispersion, rownames(y)))
# Norm
sizeFactors <- calcNormFactors(y$counts)

In [4]:
results <- glmmSeq(~ case_control + sex + ethnicity + calc_age + (1|individual_id),
                  id = "individual_id",
                  countdata = y$counts,
                  metadata = y$samples,
                  dispersion = disp,
                  sizeFactors = sizeFactors,
                  removeDuplicatedMeasures = FALSE,
                  removeSingles=FALSE,
                  progress=TRUE, cores = 24)


n = 60 samples, 33 individuals
Time difference of 8.498333 mins


In [5]:
results <- glmmQvals(results, pi0=1)


q_case_control
--------------
Not Significant     Significant 
           6435             629 

q_sex
-----
Not Significant     Significant 
           6153             911 

q_ethnicity
-----------
Not Significant     Significant 
           6391             673 

q_calc_age
----------
Not Significant     Significant 
           6644             420 


In [6]:
head(results@stats)

,Dispersion,AIC,logLik,(Intercept),case_controlPOSITIVE,sexM,ethnicityblack,ethnicityother,ethnicitywhite,calc_age,⋯,Chisq_ethnicity,Chisq_calc_age,P_case_control,P_sex,P_ethnicity,P_calc_age,q_case_control,q_sex,q_ethnicity,q_calc_age
NOC2L,0.026887873,604.8409,-293.42043,0.5688707,0.3243169,1.0083362,1.667141,0.91983028,0.43025995,0.002311535,⋯,5.265048,0.02143256,0.5167045,0.01785554,0.15338652,0.8836067,0.7782192,0.06379410,0.2228553,0.9993565
ISG15,1.201505505,634.2046,-308.10228,2.9662802,0.4951380,0.7462890,1.063126,0.59653748,0.67777359,-0.011547584,⋯,1.811781,0.26852325,0.4928229,0.20314163,0.61237462,0.6043241,0.7723508,0.21712702,0.6193006,0.9993565
TNFRSF18,1.090274053,200.5393,-91.26966,-2.7801456,-0.2275657,1.2437907,1.629526,-0.06247041,-0.03823493,0.029907772,⋯,5.354545,1.78833913,0.7263888,0.01556542,0.14760192,0.1811287,0.8478830,0.06341166,0.2190922,0.9993565
SDF4,0.074926743,795.3678,-388.68391,1.6513090,0.3175639,1.0784994,1.962467,0.99151372,0.61751316,0.006016757,⋯,7.068579,0.14288747,0.5287505,0.01174592,0.06974375,0.7054272,0.7807927,0.05967859,0.1910931,0.9993565
B3GALT6,0.205783835,329.7748,-155.88741,0.1149836,0.1094871,0.6801167,1.332210,0.17672941,0.46899560,-0.002126264,⋯,2.684337,0.01283557,0.8513612,0.16763802,0.44289558,0.9097973,0.9144010,0.18388120,0.4579028,0.9993565
UBE2J2,0.008795659,710.5910,-346.29549,0.6491027,0.1738913,0.8798714,1.379543,0.60467999,0.40480242,0.008519946,⋯,4.032669,0.33524928,0.7068704,0.02557313,0.25795801,0.5625843,0.8408400,0.06804090,0.2977476,0.9993565


In [7]:
modelData <- results@modelData
outLabels <- apply(modelData, 1, function(x) paste(x, collapse="_"))
modelData$y <- paste0('y_', outLabels)
cols1 = grep('POSITIVE', modelData$y, value = TRUE)
cols2 = grep('POSITIVE', modelData$y, value = TRUE, invert = TRUE)
LFC <- log2(rowMeans(results@predict[, cols1])+1) - log2(rowMeans(results@predict[, cols2])+1)
tmp <- data.frame(results@stats[,c('case_controlPOSITIVE', 'P_case_control', 'q_case_control')])
colnames(tmp) <- c('fixed-effects estimates', 'pval', 'qval')
tmp$LFC <- LFC
tmp <- tmp[tmp$qval < 0.05,]
tmp <- tmp[order(-tmp$`fixed-effects estimates`, tmp$qval), ]
head(tmp, 30)

,fixed-effects estimates,pval,qval,LFC
,<dbl>,<dbl>,<dbl>,<dbl>
IGHE,2.0655758,0,0,1.8071052
AC141272.1,1.9022946,0,0,1.0050284
SASS6,1.3596144,0,0,1.3798152
CDCA8,1.3409471,0,0,1.2352663
SEPT10,1.3221143,0,0,1.6097305
FBXL8,1.2557856,0,0,1.3723878
Z93241.1,1.2298445,0,0,1.2435597
KIF11,1.2121895,0,0,1.3047469
XPO1,1.1847144,0,0,1.4776921


In [8]:
tail(tmp, 30)

,fixed-effects estimates,pval,qval,LFC
,<dbl>,<dbl>,<dbl>,<dbl>
LAPTM4A,-0.1008413,5.558766e-235,7.139477e-234,-0.1378011
ATP23,-0.1117393,0.000000e+00,0.000000e+00,-0.1273476
RGS2,-0.1127026,3.847354e-10,4.397687e-09,-0.1265582
PCBD1,-0.1213554,2.964750e-174,3.733154e-173,-0.1543433
MARCH5,-0.1214197,0.000000e+00,0.000000e+00,-0.1416873
MANEA-DT,-0.1237281,5.766788e-309,7.515976e-308,-0.1482285
PDS5A,-0.1335985,0.000000e+00,0.000000e+00,-0.1775943
PBX2,-0.1476914,0.000000e+00,0.000000e+00,-0.1886985
KCNMA1,-0.1554643,1.241281e-19,1.439805e-18,-0.1034049


In [9]:
results2 <- glmmSeq(~ WHO_severity + sex + ethnicity + calc_age + (1|individual_id),
                  id = "individual_id",
                  countdata = y$counts,
                  metadata = y$samples,
                  dispersion = disp,
                  designMatrix = model.matrix(~ WHO_severity + sex + ethnicity + calc_age, data=y$samples),
                  sizeFactors = sizeFactors,
                  removeDuplicatedMeasures = FALSE,
                  removeSingles=FALSE,
                  progress=TRUE, cores = 1)


n = 60 samples, 33 individuals


boundary (singular) fit: see ?isSingular

Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive definite or contains NA values: falling back to var-cov estimated from RX”
Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive definite or contains NA values: falling back to var-cov estimated from RX”
Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive definite or contains NA values: falling back to var-cov estimated from RX”
Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive definite or contains NA values: falling back to var-cov estimated from RX”
Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finit

Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive definite or contains NA values: falling back to var-cov estimated from RX”
Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive definite or contains NA values: falling back to var-cov estimated from RX”
Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive definite or contains NA values: falling back to var-cov estimated from RX”
Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive definite or contains NA values: falling back to var-cov estimated from RX”
Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive defin

Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive definite or contains NA values: falling back to var-cov estimated from RX”
Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive definite or contains NA values: falling back to var-cov estimated from RX”
Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive definite or contains NA values: falling back to var-cov estimated from RX”
Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive definite or contains NA values: falling back to var-cov estimated from RX”
Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive defin

Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive definite or contains NA values: falling back to var-cov estimated from RX”
Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive definite or contains NA values: falling back to var-cov estimated from RX”
Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive definite or contains NA values: falling back to var-cov estimated from RX”
Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive definite or contains NA values: falling back to var-cov estimated from RX”
Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive defin

...

Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive definite or contains NA values: falling back to var-cov estimated from RX”
Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive definite or contains NA values: falling back to var-cov estimated from RX”
Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive definite or contains NA values: falling back to var-cov estimated from RX”
Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive definite or contains NA values: falling back to var-cov estimated from RX”
Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive defin

Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive definite or contains NA values: falling back to var-cov estimated from RX”
Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive definite or contains NA values: falling back to var-cov estimated from RX”
Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive definite or contains NA values: falling back to var-cov estimated from RX”
Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive definite or contains NA values: falling back to var-cov estimated from RX”
Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive defin

Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive definite or contains NA values: falling back to var-cov estimated from RX”
Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive definite or contains NA values: falling back to var-cov estimated from RX”
Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive definite or contains NA values: falling back to var-cov estimated from RX”
Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive definite or contains NA values: falling back to var-cov estimated from RX”
Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive defin

Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive definite or contains NA values: falling back to var-cov estimated from RX”
Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive definite or contains NA values: falling back to var-cov estimated from RX”
Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive definite or contains NA values: falling back to var-cov estimated from RX”
Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive definite or contains NA values: falling back to var-cov estimated from RX”
Warning message in vcov.merMod(mod, complete = FALSE):
“variance-covariance matrix computed from finite-difference Hessian is
not positive defin

Time difference of 2.449132 hours
Errors in 1 gene(s):IFIT1

In [10]:
results2 <- glmmQvals(results2, pi0=1)


q_WHO_severity
--------------
Not Significant     Significant 
           6486             577 

q_sex
-----
Not Significant     Significant 
           6379             684 

q_ethnicity
-----------
Not Significant     Significant 
           6493             570 

q_calc_age
----------
Not Significant     Significant 
           6863             200 


In [11]:
head(results2@stats)
# not sure how to convert to LFC...

,Dispersion,AIC,logLik,(Intercept),WHO_severity.L,WHO_severity.Q,WHO_severity.C,WHO_severity^4,sexM,ethnicityblack,⋯,Chisq_ethnicity,Chisq_calc_age,P_WHO_severity,P_sex,P_ethnicity,P_calc_age,q_WHO_severity,q_sex,q_ethnicity,q_calc_age
NOC2L,0.026887873,607.5264,-291.76321,1.4602359,0.5820746,0.3530443,0.20017335,0.56098366,0.8783890,1.1704455,⋯,2.616027,0.079258640,0.4246184,5.064428e-02,4.546867e-01,7.783046e-01,0.6299132,1.074272e-01,5.860315e-01,0.99991366
ISG15,1.201505505,635.6797,-305.83985,4.3939909,1.3006521,0.3116355,-0.74702119,-0.01073726,0.3478023,0.3869752,⋯,494.104835,22.556934412,0.0000000,3.996740e-68,9.041161e-107,2.040068e-06,0.0000000,5.123226e-67,1.148520e-105,0.00019212
TNFRSF18,1.090274053,202.0273,-89.01363,-3.9590794,0.3278611,0.7274737,0.00432954,-0.90240365,0.6987971,1.3912737,⋯,4.418642,3.281867515,0.3417418,2.222452e-01,2.196631e-01,7.004918e-02,0.6258156,2.486878e-01,5.504034e-01,0.99991366
SDF4,0.074926743,795.9220,-385.96099,2.7319058,0.5693644,0.2371015,0.03360662,0.88999841,1.0299305,1.4662522,⋯,4.267534,0.096568394,0.1883690,1.936002e-02,2.339869e-01,7.559872e-01,0.6258156,8.566699e-02,5.504034e-01,0.99991366
B3GALT6,0.205783835,333.0601,-154.53003,0.8760435,0.3483873,0.4258418,0.20637546,0.69083607,0.6104598,0.8473562,⋯,1.179608,0.291390327,0.6111225,2.530133e-01,7.578993e-01,5.893310e-01,0.7080061,2.764593e-01,7.823798e-01,0.99991366
UBE2J2,0.008795659,711.6285,-343.81426,1.3867010,0.3118138,0.3426396,0.32000345,0.79111518,0.8507352,0.9542757,⋯,2.021432,0.001749453,0.2586966,3.785875e-02,5.679703e-01,9.666370e-01,0.6258156,9.920434e-02,6.440157e-01,0.99991366


### run it as a job with a Rscript

```bash
Rscript --vanilla glmmSeq_jobs.R \
       -i /lustre/scratch117/cellgen/team297/kt16/COVID_imperial_renal/h5ad/df.fil3_gex_bcells_vdj_sce_B_ASC_dividing.RDS \
       -o /lustre/scratch117/cellgen/team297/kt16/COVID_imperial_renal/h5ad/glmmSeq_B_ASC_dividing.RData \
       -m 5 \
       -n 24

saving the various results as a .Rdata to open later
```